In [72]:
import os
import json
import numpy
import pandas

In [2]:
restids=pandas.read_pickle("data/restaurants.pkl")

In [3]:
td=pandas.read_pickle("data/train_data.pkl")
td2=pandas.read_pickle("data/test_data.pkl")
vd=pandas.read_pickle("data/val_data.pkl")

In [5]:
print(td.columns,td.shape)
print(td2.columns,td2.shape)
print(vd.columns,vd.shape)

Index(['business_id', 'name', 'pricerange', 'review_count', 'all_reviews',
       'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object') (86462, 9)
Index(['business_id', 'name', 'pricerange', 'review_count', 'all_reviews',
       'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object') (10809, 9)
Index(['business_id', 'name', 'pricerange', 'review_count', 'all_reviews',
       'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object') (10808, 9)


In [6]:
reviews_prepped=pandas.concat([td,td2,vd],ignore_index=True)
print(reviews_prepped.columns,reviews_prepped.shape)

Index(['business_id', 'name', 'pricerange', 'review_count', 'all_reviews',
       'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object') (108079, 9)


In [10]:
print(reviews_prepped.shape)

(108079, 9)


In [ ]:
print(reviews_prepped.iloc[0]['all_reviews'])

In [8]:
print(restids.columns)

Index(['business_id', 'name', 'city', 'stars', 'review_count', 'categories',
       'hours', 'keep'],
      dtype='object')


In [11]:
rest_revs=pandas.merge(restids,reviews_prepped,how='left',
                                 left_on='business_id',right_on='business_id')
print(rest_revs.columns)
print(rest_revs.shape)

Index(['business_id', 'name_x', 'city', 'stars', 'review_count_x',
       'categories', 'hours', 'keep', 'name_y', 'pricerange', 'review_count_y',
       'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object')
(42237, 16)


In [12]:
rest_revs=rest_revs.drop(['categories','hours','keep','pricerange','review_count_y'],axis=1)

In [13]:
rest_revs['name_y'].isna().sum()

5613

In [18]:
rem_restids=rest_revs[rest_revs['name_y'].isna()]['business_id']
print(rem_restids.shape)
print(rem_restids[:5])

(5613,)
17    X8mtoSxY8whtmbDlj0D3Aw
21    OezwOIJ0_62Mj8MLdkGRAw
32    jwHtNvHBPafiAwM92jM8yA
40    4n81G-pmC3rfhmaPsbwYKg
54    4Rp1y8L_zecjHt6AvKtekg
Name: business_id, dtype: object


In [14]:
def read_data(datafilename):
    data_file = open(datafilename,'r')
    data = []
    for line in data_file:
        data.append(json.loads(line))
    data_file.close()
    return data

In [15]:
reviews='data/yelp_academic_dataset_review.json'
raw=read_data(reviews)
df_ = pandas.json_normalize(raw,max_level=1)
print(df_.shape)
print(df_.columns)

(6685900, 9)
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')


In [19]:
%%time
revdf=df_[df_['business_id'].isin(rem_restids)]

CPU times: user 988 ms, sys: 1.13 s, total: 2.11 s
Wall time: 2.31 s


In [42]:
chkrow=revdf[revdf['business_id']=='X8mtoSxY8whtmbDlj0D3Aw']
print(chkrow)

                     review_id                 user_id  \
113192  7eg1WQOiy3H8dzNWcUntsA  pW9LvtC-hZ0PRAOUn7aZ2A   
254905  lH3WcO_PqVlyx1xcBCZLBw  Uw-v0UN_g_oFvldC7HWXMg   
374174  mIwRo2uVpjR2vXs3h-d9ug  XybrvLEZUD6lJpsy5P3gSA   
644727  9Ziz9X1WQ7I4bU4ShdNbgA  GVn_BhCNwJsQzvPy5ECYvA   

                   business_id  stars  useful  funny  cool  \
113192  X8mtoSxY8whtmbDlj0D3Aw    2.0       1      1     1   
254905  X8mtoSxY8whtmbDlj0D3Aw    3.0       0      0     0   
374174  X8mtoSxY8whtmbDlj0D3Aw    1.0       1      0     0   
644727  X8mtoSxY8whtmbDlj0D3Aw    1.0       0      1     0   

                                                     text  \
113192  It was our first night/ day in Vegas. Kinda ha...   
254905  I'm not sure where to start. This place remind...   
374174  Worst Chinese buffet I've had in town. Got a f...   
644727  too easy. DO NOT eat here. \n\nits like a big ...   

                       date               date2  
113192  2008-05-15 01:26:07 2008-05-15 01:

In [20]:
print(revdf.shape)
print(revdf.columns)

(49271, 9)
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')


In [45]:
revdf['date2']=pandas.to_datetime(revdf['date'])
rev_=revdf.sort_values(by=['useful','date2'],ascending=[False,False])
rev_sorted=rev_.groupby('business_id').head(100)
print(rev_sorted.shape)
print(rev_sorted.columns)

(47871, 10)
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'date2'],
      dtype='object')


In [46]:
chkrow=rev_sorted[rev_sorted['business_id']=='X8mtoSxY8whtmbDlj0D3Aw']
print(chkrow)

                     review_id                 user_id  \
374174  mIwRo2uVpjR2vXs3h-d9ug  XybrvLEZUD6lJpsy5P3gSA   
113192  7eg1WQOiy3H8dzNWcUntsA  pW9LvtC-hZ0PRAOUn7aZ2A   
644727  9Ziz9X1WQ7I4bU4ShdNbgA  GVn_BhCNwJsQzvPy5ECYvA   
254905  lH3WcO_PqVlyx1xcBCZLBw  Uw-v0UN_g_oFvldC7HWXMg   

                   business_id  stars  useful  funny  cool  \
374174  X8mtoSxY8whtmbDlj0D3Aw    1.0       1      0     0   
113192  X8mtoSxY8whtmbDlj0D3Aw    2.0       1      1     1   
644727  X8mtoSxY8whtmbDlj0D3Aw    1.0       0      1     0   
254905  X8mtoSxY8whtmbDlj0D3Aw    3.0       0      0     0   

                                                     text  \
374174  Worst Chinese buffet I've had in town. Got a f...   
113192  It was our first night/ day in Vegas. Kinda ha...   
644727  too easy. DO NOT eat here. \n\nits like a big ...   
254905  I'm not sure where to start. This place remind...   

                       date               date2  
374174  2008-06-28 01:13:31 2008-06-28 01:

In [24]:
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/aaksha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
stemmer = nltk.stem.WordNetLemmatizer()
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE,' ',text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE,' ',text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text_words=text.split(' ')
    #lowercase,remove urls,delete stopwords and blanks
    resultwords= []
    for word in text_words:
        if len(word)<=0: continue
        if word in STOPWORDS: continue
        if word.find('http')==0: word='url'
        resultwords.append(word)
        
    #resultwords  = [word for word in text_words if word.lower() not in STOPWORDS and len(word)>0] # delete stopwords from text
    final_words = [stemmer.lemmatize(token) for token in resultwords] #lemmatize words
    text = ' '.join(final_words) 
    return text


In [47]:
%%time
rev_sorted['text_prep']=rev_sorted['text'].apply(text_prepare)

CPU times: user 8.87 s, sys: 77.4 ms, total: 8.95 s
Wall time: 8.97 s


In [48]:
rev_sorted=rev_sorted.drop(['text'],axis=1)

In [49]:
chkrow=rev_sorted[rev_sorted['business_id']=='X8mtoSxY8whtmbDlj0D3Aw']
print(chkrow)

                     review_id                 user_id  \
374174  mIwRo2uVpjR2vXs3h-d9ug  XybrvLEZUD6lJpsy5P3gSA   
113192  7eg1WQOiy3H8dzNWcUntsA  pW9LvtC-hZ0PRAOUn7aZ2A   
644727  9Ziz9X1WQ7I4bU4ShdNbgA  GVn_BhCNwJsQzvPy5ECYvA   
254905  lH3WcO_PqVlyx1xcBCZLBw  Uw-v0UN_g_oFvldC7HWXMg   

                   business_id  stars  useful  funny  cool  \
374174  X8mtoSxY8whtmbDlj0D3Aw    1.0       1      0     0   
113192  X8mtoSxY8whtmbDlj0D3Aw    2.0       1      1     1   
644727  X8mtoSxY8whtmbDlj0D3Aw    1.0       0      1     0   
254905  X8mtoSxY8whtmbDlj0D3Aw    3.0       0      0     0   

                       date               date2  \
374174  2008-06-28 01:13:31 2008-06-28 01:13:31   
113192  2008-05-15 01:26:07 2008-05-15 01:26:07   
644727  2009-02-10 20:20:11 2009-02-10 20:20:11   
254905  2008-01-16 22:49:01 2008-01-16 22:49:01   

                                                text_prep  
374174  worst chinese buffet town got flyer mail pictu...  
113192  first night da

In [50]:
%%time
rev_sorted['all_reviews']=rev_sorted.groupby(['business_id'])['text_prep'].transform(lambda x: '\t'.join(x))
rev_sorted['stars_sum']=rev_sorted.groupby(['business_id'])['stars'].transform('sum')
rev_sorted['useful_sum']=rev_sorted.groupby(['business_id'])['useful'].transform('sum')
rev_sorted['funny_sum']=rev_sorted.groupby(['business_id'])['funny'].transform('sum')
rev_sorted['cool_sum']=rev_sorted.groupby(['business_id'])['cool'].transform('sum')
rev_sorted=rev_sorted.drop(['stars','useful','funny','cool','date','date2','review_id','user_id','text_prep'],axis=1)
rev_sorted.drop_duplicates(inplace=True)
print(rev_sorted.shape)
print(rev_sorted.columns)

(5613, 6)
Index(['business_id', 'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum',
       'cool_sum'],
      dtype='object')
CPU times: user 1.94 s, sys: 2.7 s, total: 4.65 s
Wall time: 4.91 s


In [51]:
chkrow=rev_sorted[rev_sorted['business_id']=='X8mtoSxY8whtmbDlj0D3Aw']
print(chkrow)

                   business_id  \
374174  X8mtoSxY8whtmbDlj0D3Aw   

                                              all_reviews  stars_sum  \
374174  worst chinese buffet town got flyer mail pictu...        7.0   

        useful_sum  funny_sum  cool_sum  
374174           2          2         1  


In [29]:
print(rest_revs.columns)
print(rest_revs.shape)

Index(['business_id', 'name_x', 'city', 'stars', 'review_count_x', 'name_y',
       'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object')
(42237, 11)


In [54]:
#rest_revs.set_index('business_id',inplace=True)
#rev_sorted.set_index('business_id',inplace=True)
#rest_revs.reset_index(inplace=True)
print(rest_revs.columns)

Index(['business_id', 'name_x', 'city', 'stars', 'review_count_x', 'name_y',
       'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object')


In [36]:
print(rev_sorted.columns)
print(rev_sorted.iloc[0])

Index(['all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'], dtype='object')
all_reviews    u two kind people jimmy john jj lover jj hater...
stars_sum                                                   34.0
useful_sum                                                   221
funny_sum                                                    169
cool_sum                                                     225
Name: hT4S-jtRapUap_kzygb_Hw, dtype: object


In [84]:
counter=0
for restid in rem_restids:
    #print(restid)
    #new_data=rev_sorted[restid]
    new_data=rev_sorted[rev_sorted['business_id']==restid]
    #print(new_data)
    old_data_index=rest_revs[rest_revs['business_id']==restid].index.to_list()[0]
    #print(old_data_index)
    old_data=rest_revs.loc[old_data_index]
    #print(old_data)
    rest_revs.at[old_data_index,'all_reviews']=new_data.iloc[0]['all_reviews']
    rest_revs.at[old_data_index,'stars_sum']=new_data.iloc[0]['stars_sum']
    rest_revs.at[old_data_index,'useful_sum']=new_data.iloc[0]['useful_sum']
    rest_revs.at[old_data_index,'funny_sum']=new_data.iloc[0]['funny_sum']
    rest_revs.at[old_data_index,'cool_sum']=new_data.iloc[0]['cool_sum']
    counter+=1
    if counter%1000==0:
        print("Done "+str(counter))
    #print(rest_revs.loc[old_data_index])
    #break

Done 1000
Done 2000
Done 3000
Done 4000
Done 5000


In [68]:
print(rest_revs['stars_sum'].isna().sum())
print(rest_revs.shape)
print(rest_revs.columns)
rest_revs=rest_revs.drop(['name_y'],axis=1)
rest_revs=rest_revs.rename({'stars':'business_stars','review_count_x':'review_count',
                            'name_x':'name'},axis=1)
print(rest_revs.shape)
print(rest_revs.columns)

0
(42237, 10)
Index(['business_id', 'name_x', 'city', 'business_stars', 'review_count',
       'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object')
(42237, 10)
Index(['business_id', 'name', 'city', 'business_stars', 'review_count',
       'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum'],
      dtype='object')


In [83]:
#for debugging, skip
for i in range(rest_revs.shape[0]):
    t=rest_revs.iloc[i]['all_reviews']
    if type(t)!=type('abc'):
        print(rest_revs.iloc[i])
    break

<class 'str'>


In [69]:
#load embeddings
dims=50
dimheader=[]
for i in range(dims):
    dimheader.append(str(i))
print(dimheader)
embeddingsdf=pandas.read_table('models/embeddings/model.tsv',header=None,names=['word']+dimheader)
print(embeddingsdf.shape)
#create numpy array of embeddings
onlyembeddings=embeddingsdf.drop(['word'],axis=1).to_numpy()
#index vocab for faster processing
word2index={}
index2word={}
for i in range(embeddingsdf.shape[0]):
    word=embeddingsdf.iloc[i]['word']
    word2index[word]=i
    index2word[i]=word

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49']
(237728, 51)


In [76]:
def construct_user_vector(ut):  
    utsents=ut.split('\t')
    utwords=[]
    for sent in utsents:
        words=sent.split(' ')
        utwords+=words
    utids=[]
    for word in utwords:
        try:
            wi=word2index[word]
        except:
            continue
        utids.append(wi)
    embeddingmatrix=onlyembeddings[[utids],:]
    embeddingmatrix=numpy.reshape(embeddingmatrix,(len(utids),onlyembeddings.shape[1]))
    user_embedding=numpy.mean(embeddingmatrix,axis=0)
    return(user_embedding)

In [ ]:
t=rest_revs.iloc[0]['all_reviews']
print(rest_revs.at[374172,'all_reviews'])
# tv=construct_user_vector(t)
# print(tv)

In [85]:
rest_revs['business_vector']=rest_revs['all_reviews'].apply(construct_user_vector)
print(rest_revs.shape)
print(rest_revs.columns)

(42237, 11)
Index(['business_id', 'name', 'city', 'business_stars', 'review_count',
       'all_reviews', 'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum',
       'business_vector'],
      dtype='object')


In [86]:
rest_trove=rest_revs.drop(['all_reviews'],axis=1)
print(rest_trove.shape)
print(rest_trove.columns)

(42237, 10)
Index(['business_id', 'name', 'city', 'business_stars', 'review_count',
       'stars_sum', 'useful_sum', 'funny_sum', 'cool_sum', 'business_vector'],
      dtype='object')


In [87]:
rest_trove.to_pickle('models/restaurant_trove.pkl')